In [1]:
import numpy as np
import csv
from pathlib import Path
import scipy.io
from scipy.signal import resample_poly, butter, filtfilt
import pandas as pd

In [11]:
from pathlib import Path
files = []
example_mat = Path("example.mat")
directory = Path('raw_data/ball')
output_dir = Path('mat_data/filtered/ball')
for file in directory.iterdir():
    if file.is_file():
        files.append(file)

In [3]:
def extract_data(file_path):
    header = []
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line == "Data as Time Sequence:":
                break
            if line:  
                header.append(line)
        
        for line in file:
            line = line.strip()
            if line:  
                try:
                    value = float(line)
                    data.append(value)
                except ValueError:
                    continue  
    
    return header, data

In [4]:
def filter(data, fs = 9000, cutoff = 30):                    
    acc = data
    time = np.arange(len(acc)) / fs
    order = 4  
    b, a = butter(N=order, Wn=cutoff/(fs/2), btype='high')
    acc_filtered = filtfilt(b, a, acc)
    return acc_filtered 

In [ ]:
def bandstop_filter(data, fs=9000, lowcut=10, highcut=40, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandstop')
    filtered_data = filtfilt(b, a, data)
    print(filtered_data)
    return filtered_data


In [33]:
for file_path in files: 
    mat_source = scipy.io.loadmat(example_mat)
    header, data = extract_data(file_path)
    filtered_data = bandstop_filter(data) #filter
    mat_source["X118_DE_time"] = np.array([[x] for x in filtered_data])
    file_name = file_path.name.split(".")[:-1][0]
    scipy.io.savemat(str(output_dir) + "/" + file_name + ".mat", mat_source)

[ 0.44322898  0.4471775   0.44154178 ... -0.02647729 -0.0264396
 -0.03030928]
[ 2.11863482  2.11475148  2.13477952 ... -0.02267656  0.00428924
 -0.01852891]
[-0.63592317 -0.66890093 -0.67099042 ...  0.54801221  0.52838572
  0.53001684]
[-1.03236608 -1.048182   -1.0678466  ... -0.05385931 -0.01158907
 -0.04967089]
[-10.69476035 -10.76909543 -10.51762393 ...  -0.60017934  -0.52693146
  -0.48975813]
[-7.13918205e-04 -6.41035443e-04 -5.88209325e-04 ...  1.23922743e+00
  1.24207552e+00  1.24320160e+00]
